<a href="https://colab.research.google.com/github/seonghunYang/big_contest2020/blob/master/model_lstm_%ED%8C%A8%ED%84%B4%EB%B0%98%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
r_team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/2차_최종_최근_팀타자.csv', encoding='utf-8-sig')
r_team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/2차_최종_최근_팀투수.csv', encoding='utf-8-sig')
vs_r_team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/2차_최종_최근_상대_팀타자.csv', encoding='utf-8-sig')
vs_r_team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/2차_최종_최근_상대_팀투수.csv', encoding='utf-8-sig')
team_data = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_공통.csv', encoding='utf-8-sig')
sr_team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/모델/선발_팀투수_최종.csv', encoding='utf-8-sig')

In [4]:
r_team_pitcher = r_team_pitcher[["게임키", '일자', '자책점', '투구수', '실책', '결과', '득점권WHIP', '단타', '홈런', '2루타', '희비', '이닝*3',
                                 "3루타", "사구", '도루실패', '홀드', '폭투', '상대팀코드']]

In [5]:
r_team_pitcher.columns = ['게임키', '일자','자책점', '투구수', '실책', '결과', '득점권WHIP', '단타', '홈런', '2루타', '희비',
       '이닝*3', '3루타', '사구', '도루실패', '홀드', '폭투', '팀코드']

In [6]:
r_team_hitter = r_team_hitter[["게임키", '득점권안타', "홈런", "희비", "4구", "잔루", "타수", "사구", "단타", "2루타", "병살타", "도루실패", "3루타", "득점권타수",
                               "초말", "희타", "삼진", "팀코드", "시즌", '득점']]

In [8]:
r_team_hitter = pd.merge(r_team_hitter, r_team_pitcher, on=['게임키', '팀코드'])

In [9]:
def runMinus(row):
  previous = r_team_hitter[(r_team_hitter['팀코드'] == row['팀코드']) & (r_team_hitter['일자'] < row['일자']) & (r_team_hitter['시즌'] == row['시즌'])].tail(1)
  if len(previous) == 0:
    row['자책점차이'] = 0
    return row
  previous = previous.sum()
  runM = row['자책점'] - previous['자책점']
  row['자책점차이'] = runM
  return row
r_team_hitter = r_team_hitter.apply(runMinus, axis=1)

In [ ]:
r_team_hitter[r_team_hitter['팀코드'] == 'LG']

,게임키,득점권안타,홈런_x,희비_x,4구,잔루,타수,사구_x,단타_x,2루타_x,병살타,도루실패_x,3루타_x,득점권타수,초말,희타,삼진,팀코드,시즌,득점,일자,자책점,투구수,실책,결과,득점권WHIP,단타_y,홈런_y,2루타_y,희비_y,이닝*3,3루타_y,사구_y,도루실패_y,홀드,폭투,자책점차이
0,20160401HHLG0,4,1,0,4,8,42,0,6,2,0,1,0,12,B,1,11,LG,2016,5,2016-04-01,4,196,0,L,1.500,6,1,2,0,34,0,0,1,0,1,0
10,20160402HHLG0,3,1,0,8,11,40,1,8,2,0,2,1,10,B,0,9,LG,2016,8,2016-04-02,8,222,1,L,4.000,8,1,2,0,30,1,1,2,0,2,4
25,20160405LGHT0,1,0,0,1,7,34,0,7,0,0,0,0,5,T,0,9,LG,2016,1,2016-04-05,1,137,0,W,0.750,7,0,0,0,27,0,0,0,2,0,-7
43,20160407LGHT0,4,2,0,4,8,38,1,8,1,0,0,0,12,T,0,9,LG,2016,8,2016-04-07,8,162,0,L,1.875,8,2,1,0,27,0,1,0,0,0,7
57,20160408LGSK0,1,1,0,2,4,33,1,3,0,1,0,0,5,T,0,4,LG,2016,2,2016-04-08,2,121,0,W,0.750,3,1,0,0,30,0,1,0,0,0,-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6286,20200715LGLT0,7,1,0,3,8,40,0,9,3,0,0,1,17,T,1,8,LG,2020,9,2020-07-15,9,157,0,L,2.400,9,1,3,0,27,1,0,0,0,0,9
6296,20200716LGLT0,6,1,1,5,8,38,1,14,1,3,0,0,8,T,0,4,LG,2020,10,2020-07-16,10,183,0,W,6.750,14,1,1,1,27,0,1,0,2,0,1
6301,20200717HHLG0,3,3,0,6,8,32,2,5,2,0,1,0,9,B,0,10,LG,2020,8,2020-07-17,8,165,0,L,1.714,5,3,2,0,24,0,2,1,0,1,-2
6311,20200718HHLG0,1,1,0,4,5,27,0,4,0,0,0,0,7,B,1,8,LG,2020,3,2020-07-18,2,130,0,L,0.857,4,1,0,0,24,0,0,0,0,0,-6


In [ ]:
r_team_hitter = pd.concat([r_team_hitter, r_team_pitcher], axis=1)
r_team_hitter.head(20)

,득점권안타,홈런,홈런,홈런,홈런,희비,희비,희비,희비,4구,잔루,타수,사구,사구,사구,사구,단타,단타,단타,단타,2루타,2루타,2루타,2루타,병살타,도루실패,도루실패,도루실패,도루실패,3루타,3루타,3루타,3루타,득점권타수,초말,희타,삼진,팀코드,시즌,득점,자책점,투구수,실책,결과,득점권WHIP,단타,홈런,2루타,희비,이닝*3,3루타,사구,도루실패,홀드,폭투
0,4,1,0,0,0,0,0,0,0,4,8,42,0,0,0,0,6,11,11,11,2,2,2,2,0,1,0,0,0,0,0,0,0,12,B,1,11,LG,2016,5,4,184,0,W,0.643,11,0,2,0,36,0,0,0,0,0
1,3,0,1,1,1,0,0,0,0,3,12,46,0,0,0,0,11,6,6,6,2,2,2,2,1,0,1,1,1,0,0,0,0,15,T,3,10,HH,2016,4,4,196,0,L,1.500,6,1,2,0,34,0,0,1,0,1
2,1,2,1,1,1,0,0,0,0,5,7,30,0,1,1,1,5,4,4,4,2,3,3,3,1,0,0,0,0,0,0,0,0,7,B,1,9,NC,2016,5,4,143,0,W,1.333,4,1,3,0,27,0,1,0,0,2
3,1,1,2,2,2,0,0,0,0,3,7,34,1,0,0,0,4,5,5,5,3,2,2,2,1,0,0,0,0,0,0,0,0,10,T,0,10,HT,2016,4,5,147,0,L,0.500,5,2,2,0,24,0,0,0,0,0
4,3,1,2,2,2,0,1,1,1,0,5,36,0,1,1,1,4,9,9,9,5,0,0,0,1,0,1,1,1,0,0,0,0,8,B,0,7,SK,2016,4,8,151,0,L,1.000,9,2,0,1,27,0,1,1,0,0
5,2,2,1,1,1,1,0,0,0,4,7,36,1,0,0,0,9,4,4,4,0,5,5,5,0,1,0,0,0,0,0,0,0,7,T,0,7,KT,2016,8,4,147,0,W,1.800,4,1,5,0,27,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0,5,8,30,0,0,0,0,5,9,9,9,1,2,2,2,0,2,0,0,0,0,0,0,0,5,B,0,6,WO,2016,1,1,145,0,L,0.250,9,0,2,0,27,0,0,0,0,0
7,1,0,0,0,0,0,1,1,1,2,10,37,0,0,0,0,9,5,5,5,2,1,1,1,2,0,2,2,2,0,0,0,0,14,T,0,6,LT,2016,2,0,138,1,W,0.000,5,0,1,1,27,0,0,2,4,0
8,1,0,2,2,2,0,1,1,1,2,7,32,1,1,1,1,8,6,6,6,0,0,0,0,2,1,1,1,1,0,0,0,0,4,B,0,6,SS,2016,1,4,147,1,L,0.375,6,2,0,1,27,0,1,1,0,0
9,1,2,0,0,0,1,0,0,0,3,5,31,1,1,1,1,6,8,8,8,0,0,0,0,1,1,1,1,1,0,0,0,0,7,T,1,3,OB,2016,5,1,146,0,W,1.000,8,0,0,0,27,0,1,1,1,1


In [10]:
# nc = 0  0 0 0 0 0 0 
team_code = ['OB', 'WO', 'HT', "LG", 'SS', 'KT', 'LT', 'SK', "HH"]
def team_encoding(row, team_code):
  if row['팀코드'] == team_code:
    row[team_code] = 1
  else:
    if row['팀코드'] != "NC":
      row[team_code] = 0
  return row

def createHome(row):
  if row['초말'] == 'B':
    row['초말'] = 1
  else:
    row['초말'] = 0
  return row

In [11]:
# 2016 = 0  0 0 0 0 0 0 
year_code = [2017, 2018, 2019, 2020]
def year_encoding(row, year_code):
  if row['시즌'] == year_code:
    row[str(year_code)] = 1
  else:
    if row['시즌'] != 2016:
      row[str(year_code)] = 0
  return row

In [12]:
for team in team_code:
  r_team_hitter = r_team_hitter.apply(team_encoding, axis=1, team_code=team)
  
r_team_hitter = r_team_hitter.apply(createHome, axis=1)

In [13]:
for year in year_code:
  r_team_hitter = r_team_hitter.apply(year_encoding, axis=1, year_code=year)

In [14]:
r_team_hitter = r_team_hitter.fillna(0)

In [ ]:
r_team_hitter.columns

Index(['2017', '2018', '2019', '2020', '2루타_x', '2루타_y', '3루타_x', '3루타_y',
       '4구', 'HH', 'HT', 'KT', 'LG', 'LT', 'OB', 'SK', 'SS', 'WO', '게임키', '결과',
       '단타_x', '단타_y', '도루실패_x', '도루실패_y', '득점', '득점권WHIP', '득점권안타', '득점권타수',
       '병살타', '사구_x', '사구_y', '삼진', '시즌', '실책', '이닝*3', '자책점', '잔루', '초말',
       '타수', '투구수', '팀코드', '폭투', '홀드', '홈런_x', '홈런_y', '희비_x', '희비_y', '희타'],
      dtype='object')

In [36]:
data = r_team_hitter[['2017', '2018', '2019', '2020', '2루타_x', '3루타_x', '4구', 'HH', 'HT', 'KT',
       'LG', 'LT', 'OB', 'SK', 'SS', 'WO', '단타_x', '도루실패_x', '득점', '득점권안타',
       '득점권타수', '병살타', '사구_x', '삼진', '잔루', '초말', '타수', '홈런_x', '희비_x', '자책점', '자책점차이',
       '희타']].copy()

In [37]:
data = data[data['OB'] == 1]

In [38]:
x_data = data[['2017', '2018', '2019', '2020', '2루타_x', '3루타_x', '4구', 'HH', 'HT', 'KT',
       'LG', 'LT', 'OB', 'SK', 'SS', 'WO', '단타_x', '도루실패_x', '득점권안타',
       '득점권타수', '병살타', '사구_x', '삼진', '잔루', '초말', '타수', '홈런_x', '희비_x', '자책점차이', '자책점',
       '희타']]

y_data = data[['자책점']].iloc[20:]

In [39]:
x_data = x_data.reset_index()
y_data = y_data.reset_index()
del x_data['index']
del y_data['index']

In [40]:
y_data

,자책점
0,4
1,1
2,6
3,3
4,6
...,...
604,7
605,4
606,2
607,6


In [45]:
x_data_value = x_data.iloc[0: 0+20].values.reshape(-1,20,31)
for idx in range(1, 609):
  seq = x_data.iloc[idx: idx+20].values
  x_data_value = np.append(x_data_value, seq.reshape(-1,20,31), axis=0)

In [46]:
x_data_value.shape

(609, 20, 31)

In [22]:
x_data_value = x_data.iloc[0: 0+20].values.reshape(-1,20,31)
for idx in range(1, 6311):
  seq = x_data.iloc[idx: idx+20].values
  x_data_value = np.append(x_data_value, seq.reshape(-1,20,31), axis=0)

In [60]:
y_data = y_data.values

In [61]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x_data_value, y_data, test_size = 0.3 ,shuffle=False
)

In [ ]:
from sklearn.preprocessing import RobustScaler
robustScaler = RobustScaler()
print(robustScaler.fit(x_train.reshape(-1, 29)))
x_train = robustScaler.transform(x_train.reshape(-1, 29))
x_train = x_train.reshape(-1, 5, 29)
from sklearn.preprocessing import RobustScaler
robustScaler = RobustScaler()
print(robustScaler.fit(x_test.reshape(-1, 29)))
x_test = robustScaler.transform(x_test.reshape(-1, 29))
x_test = x_test.reshape(-1, 5, 29)


RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
             with_scaling=True)
RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
             with_scaling=True)


In [62]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras import metrics
model = Sequential()

model.add(LSTM(1, input_shape=(20 , 31), activation= 'relu'))
# model.add(LSTM(1, activation= 'relu'))

from keras import optimizers

adam = optimizers.Adam(lr=0.005)


model.compile(loss='mse', optimizer = 'adam', metrics=[metrics.mse])

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=20)
model.fit(x_train, y_train, validation_split=0.2, verbose = 1,
          batch_size=1, epochs=100, callbacks = [early_stopping])

loss, accuracy = model.evaluate(x_test, y_test, batch_size=1)
print('lose: ', loss)
print('mse: ', accuracy)

y_pred = model.predict(x_test)

Epoch 1/100
340/340 [==============================] - 2s 5ms/step - loss: 295.4798 - mean_squared_error: 295.4798 - val_loss: 35.1465 - val_mean_squared_error: 35.1465
Epoch 2/100
340/340 [==============================] - 1s 4ms/step - loss: 33.6002 - mean_squared_error: 33.6002 - val_loss: 26.1328 - val_mean_squared_error: 26.1328
Epoch 3/100
340/340 [==============================] - 1s 4ms/step - loss: 23.3944 - mean_squared_error: 23.3944 - val_loss: 16.3949 - val_mean_squared_error: 16.3949
Epoch 4/100
340/340 [==============================] - 2s 4ms/step - loss: 16.4956 - mean_squared_error: 16.4956 - val_loss: 12.7843 - val_mean_squared_error: 12.7843
Epoch 5/100
340/340 [==============================] - 1s 4ms/step - loss: 14.5380 - mean_squared_error: 14.5380 - val_loss: 13.8515 - val_mean_squared_error: 13.8515
Epoch 6/100
340/340 [==============================] - 1s 4ms/step - loss: 14.1654 - mean_squared_error: 14.1654 - val_loss: 12.0102 - val_mean_squared_error: 12.0

In [ ]:
y_pred_train = model.predict(x_train)

In [64]:
for i in range(len(y_pred)):
  print(y_pred[i], y_test[i])

[4.6967473] [9]
[4.7210956] [9]
[5.167249] [3]
[5.202862] [6]
[5.3966246] [9]
[3.351194] [7]
[0.06218695] [8]
[1.2367342] [0]
[6.2480416] [1]
[5.4031134] [4]
[4.326797] [7]
[1.4822516] [2]
[6.797607] [10]
[5.878534] [3]
[4.8379126] [1]
[4.8117027] [1]
[4.9173937] [11]
[5.384291] [12]
[5.6908135] [3]
[5.4653907] [3]
[5.2293124] [3]
[4.538838] [1]
[5.6570845] [3]
[6.2574353] [5]
[5.670506] [7]
[4.225706] [1]
[4.6427493] [1]
[4.7406077] [3]
[5.1424828] [7]
[5.2569137] [1]
[5.032945] [3]
[4.84137] [3]
[5.611257] [1]
[4.8502345] [2]
[4.151974] [3]
[4.7140326] [7]
[4.1175838] [7]
[5.880044] [4]
[5.4924674] [3]
[5.0738363] [7]
[5.0922256] [4]
[5.2323093] [0]
[4.7784023] [0]
[5.259831] [9]
[4.067513] [6]
[4.5678153] [6]
[4.8894134] [3]
[3.8025854] [5]
[2.8290145] [10]
[2.723692] [1]
[6.3271747] [14]
[5.12745] [1]
[2.7451053] [0]
[5.3139215] [2]
[4.454531] [2]
[5.695286] [9]
[5.3898807] [2]
[4.746411] [0]
[5.0218196] [0]
[4.9401784] [3]
[4.2276835] [3]
[4.7817755] [4]
[5.148898] [3]
[6.5402884]

In [65]:
y_test = pd.DataFrame(y_test)
y_test

,0
0,9
1,9
2,3
3,6
4,9
...,...
178,7
179,4
180,2
181,6


In [ ]:
y_test[0]

0       5
1       5
2       5
3       7
4       2
       ..
1893    2
1894    4
1895    8
1896    4
1897    3
Name: 0, Length: 1898, dtype: int64

In [66]:
y_pred = pd.DataFrame(y_pred)
y_pred

,0
0,4.696747
1,4.721096
2,5.167249
3,5.202862
4,5.396625
...,...
178,5.319273
179,5.992395
180,5.236189
181,4.757068


In [ ]:
y_pred_train = pd.DataFrame(y_pred_train)
y_train = pd.DataFrame(y_train)

In [67]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test.index, y=y_test[0],
                            mode='lines+markers'))
fig.add_trace(go.Scatter(x=y_pred.index, y=y_pred[0],
                            mode='lines+markers'))
fig.update_layout(
    {
        "title": {
            "text": "a",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "showlegend": True,
        "xaxis": {
            "title": "일자",
            "showticklabels":True,
            
            
        },
        "yaxis": {
            "title": "수치"
        },

    }
)
fig.show()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test.index, y=y_test[0],
                            mode='lines+markers'))
fig.add_trace(go.Scatter(x=y_pred.index, y=y_pred[0],
                            mode='lines+markers'))
fig.update_layout(
    {
        "title": {
            "text": "a",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "showlegend": True,
        "xaxis": {
            "title": "일자",
            "showticklabels":True,
            
            
        },
        "yaxis": {
            "title": "수치"
        },

    }
)
fig.show()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_train.index, y=y_train[0],
                            mode='lines+markers'))
fig.add_trace(go.Scatter(x=y_pred_train.index, y=y_pred_train[0],
                            mode='lines+markers'))
fig.update_layout(
    {
        "title": {
            "text": "a",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "showlegend": True,
        "xaxis": {
            "title": "일자",
            "showticklabels":True,
            
            
        },
        "yaxis": {
            "title": "수치"
        },

    }
)
fig.show()